AttributeError: 'Ticker' object has no attribute 'tickers'

In [33]:
import pandas as pd
import yfinance as yf
import datetime

In [ ]:
# the objective is

In [8]:
ticker = "ALO.PA"
df = yf.download(ticker,start="2020-01-01")

[*********************100%***********************]  1 of 1 completed


In [9]:
df.head(20)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,42.270000,42.790001,42.240002,42.580002,41.451309,351895
2020-01-03,42.220001,42.639999,42.209999,42.639999,41.509720,337753
2020-01-06,42.439999,42.590000,41.849998,42.220001,41.100857,449260
2020-01-07,42.230000,42.720001,42.200001,42.500000,41.373428,411762
2020-01-08,43.119999,43.560001,43.000000,43.310001,42.161961,765403
2020-01-09,43.599998,43.720001,42.980000,43.169998,42.025669,578042
2020-01-10,43.230000,43.480000,43.060001,43.090000,41.947792,466355
2020-01-13,43.279999,43.939999,43.099998,43.549999,42.395596,672950
2020-01-14,43.590000,44.139999,43.290001,44.040001,42.872608,647802


In [10]:
df.shape

(925, 6)

In [18]:
stock = yf.Ticker(ticker)

In [28]:
div = stock.dividends
print(f"dividend {div}")

dividend Date
2006-05-15 00:00:00+02:00    0.226018
2007-07-05 00:00:00+02:00    0.370902
2008-06-26 00:00:00+02:00    0.741804
2009-06-25 00:00:00+02:00    1.038526
2010-06-24 00:00:00+02:00    1.149796
2011-06-30 00:00:00+02:00    0.574898
2012-06-28 00:00:00+02:00    0.741804
2013-07-04 00:00:00+02:00    0.778894
2017-07-07 00:00:00+02:00    0.231814
2018-07-20 00:00:00+02:00    0.324539
2019-07-15 00:00:00+02:00    5.099902
2021-08-02 00:00:00+02:00    0.250000
2022-07-20 00:00:00+02:00    0.250000
2023-07-17 00:00:00+02:00    0.250000
Name: Dividends, dtype: float64


In [36]:
def getDividendYield(stockTicker:str, currentYear:int=None):
    stock = yf.Ticker(stockTicker)
    if not currentYear:
        currentYear=datetime.datetime.now().year

    currentDiv = stock.dividends[stock.dividends.index.year == currentYear][0]
    currentPrice = stock.history(period="1d")["Close"][0]
    divYield = currentDiv / currentPrice
    print(f"The dividendYield of {stockTicker} at year {currentYear} is: {divYield}")


In [35]:
getDividendYield(ticker)

The dividendYield of year 2023 is: 0.009531071458748696


In [37]:
getDividendYield("OR.PA")

The dividendYield of OR.PA at year 2023 is: 0.014778325123152709


In [39]:
getDividendYield("SCR.PA")

The dividendYield of SCR.PA at year 2023 is: 0.05005363011687881


In [70]:
def aroundDateToWeekDay(inDate: datetime.datetime) -> datetime.datetime:
    """
    This function takes a date, if it's not a week day, convert it to the nearest week day date
    :param inDate:
    :type inDate:
    :return:
    :rtype:
    """
    dayOfWeek = inDate.weekday()
    # 0~4 is week day, 5,6 are saturday and sunday
    if dayOfWeek == 5:
        return inDate - datetime.timedelta(days=1)
    elif dayOfWeek == 6:
        return inDate - datetime.timedelta(days=2)
    else:
        return inDate


def getPriceOfDate(stock, inDate: datetime.datetime) -> float:
    """
    This function takes a stock, and a date, returns the closing price of the stock at the given day
    :param stock:
    :type stock:
    :param inDate:
    :type inDate:
    :return:
    :rtype:
    """
    price = stock.history(start=inDate, end=inDate + datetime.timedelta(days=1))
    if price.empty:
        getPriceOfDate(stock, inDate - datetime.timedelta(days=1))
    else:
        return price


def getHistoricalReturns(oldPrice: float, newPrice: float) -> float:
    """
    This function returns the historical returns of two giving price.
    :param oldPrice:
    :type oldPrice:
    :param newPrice:
    :type newPrice:
    :return:
    :rtype:
    """
    return (newPrice - oldPrice) / oldPrice * 100


def processDate(dateArg) -> datetime.date:
    """
    Convert a date argument into datetime.date type, which can be either a string or a datetime object.
    :param dateArg:
    :type dateArg:
    :return:
    :rtype:
    """

    if isinstance(dateArg, str):
        try:
            # Attempt to convert string to datetime
            processedDate = datetime.datetime.strptime(dateArg, '%Y-%m-%d').date()
        except ValueError:
            raise ValueError("Invalid date string format. Please use 'YYYY-MM-DD'.")
    elif isinstance(dateArg, datetime.date):
        processedDate = dateArg
    else:
        raise ValueError("Invalid date argument. Please provide a string or datetime object.")

    return processedDate


def getPriceEvolution(stockTicker: str, startDate, endDate=None) -> float:
    """
    This function takes a stock ticker, and two dates, then it shows the price evolution of the stock. It returns
    only the HistoricalReturns of the stock
    :param stockTicker:
    :type stockTicker:
    :param startDate:
    :type startDate:
    :param endDate:
    :type endDate:
    :return:
    :rtype:
    """
    # get the stock
    stock = yf.Ticker(stockTicker)
    # convert the start date (str or datetime) to datetime
    startDate = processDate(startDate)
    # convert the date to a weekday (if the date is a saturday or sunday)
    startDate = aroundDateToWeekDay(startDate)

    # if end date is none
    if not endDate:
        endDate = datetime.datetime.now().date()
    else:
        endDate = processDate(endDate)
    endDate = aroundDateToWeekDay(endDate)

    oldPrice = getPriceOfDate(stock, startDate)['Close'][0]
    newPrice = getPriceOfDate(stock, endDate)['Close'][0]
    print(f"old price: {oldPrice}, at {startDate}")
    print(f"new price: {newPrice}, at {endDate}")
    hr = getHistoricalReturns(oldPrice, newPrice)
    print(f"historical returns: {hr}")
    return hr


def getStockReturnsSinceDays(stockTicker: str, days: int):
    endDate = datetime.datetime.now().date()
    startDate = endDate - datetime.timedelta(days=days)
    getPriceEvolution(stockTicker, startDate, endDate)


def getLastWeekReturns(stockTicker: str):
    getStockReturnsSinceDays(stockTicker, 7)


def getLastMonthReturns(stockTicker: str):
    getStockReturnsSinceDays(stockTicker, 30)


def getLastThreeMonthReturns(stockTicker: str):
    getStockReturnsSinceDays(stockTicker, 90)


def getLastYearReturns(stockTicker: str):
    getStockReturnsSinceDays(stockTicker, 365)

In [69]:
startDate="2022-06-04"
endDate = "2023-06-04"
getPriceEvolution("SCR.PA",startDate, endDate)

<class 'datetime.datetime'>


AttributeError: 'str' object has no attribute 'weekday'

In [74]:
axa = "CS.PA"

getLastWeekReturns(axa)
getLastMonthReturns(axa)
getLastThreeMonthReturns(axa)
getLastYearReturns(axa)

old price: 26.915000915527344, at 2023-08-03
new price: 28.280000686645508, at 2023-08-10
historical returns: 5.071520433538947
old price: 25.815000534057617, at 2023-07-11
new price: 28.280000686645508, at 2023-08-10
historical returns: 9.548712382693259
old price: 26.94499969482422, at 2023-05-12
new price: 28.280000686645508, at 2023-08-10
historical returns: 4.954540756880117
old price: 22.732149124145508, at 2022-08-10
new price: 28.280000686645508, at 2023-08-10
historical returns: 24.40531043590249


In [72]:
scor = "SCR.PA"

getLastWeekReturns(scor)
getLastMonthReturns(scor)
getLastThreeMonthReturns(scor)
getLastYearReturns(scor)

old price: 27.399999618530273, at 2023-08-03
new price: 28.850000381469727, at 2023-08-10
historical returns: 5.291973661046462
old price: 26.799999237060547, at 2023-07-11
new price: 28.850000381469727, at 2023-08-10
historical returns: 7.6492582192850325
old price: 24.31087875366211, at 2023-05-12
new price: 28.850000381469727, at 2023-08-10
historical returns: 18.671154069755126
old price: 16.318971633911133, at 2022-08-10
new price: 28.850000381469727, at 2023-08-10
historical returns: 76.78810300471923


In [73]:
getDividendYield(axa)

The dividendYield of CS.PA at year 2023 is: 0.06010252816584472


In [75]:
SG="GLE.PA"
stock = yf.Ticker(ticker)

In [76]:
print(stock)

yfinance.Ticker object <ALO.PA>


In [82]:
getPriceEvolution(SG, "2020-06-16")

old price: 12.65329647064209, at 2020-06-16
new price: 25.655000686645508, at 2023-08-10
historical returns: 102.75349389125353


102.75349389125353

In [83]:
getDividendYield(SG)

The dividendYield of GLE.PA at year 2023 is: 0.06632852343601686
